In [1]:
%%capture
import waymax
import numpy as np
import mediapy
from tqdm import tqdm
import dataclasses

import tensorflow as tf

from waymax import config as _config
from waymax import dataloader
from waymax import datatypes
from waymax import visualization

2024-10-03 14:37:15.480744: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 14:37:15.503440: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 14:37:15.873310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Configurations

In [2]:
max_controlled_objects = 32

Taking files from: 
```
'gs://waymo_open_dataset_motion_v_1_1_0/uncompressed/tf_example/validation/validation_tfexample.tfrecord@150'
```
[link](https://github.com/waymo-research/waymax/blob/720f9214a9bf79b3da7926497f0cd0468ca3e630/waymax/config.py#L269)

In [3]:
data_config = dataclasses.replace(
    _config.WOD_1_1_0_VALIDATION, 
    max_num_objects=32
)

data_iter = dataloader.simulator_state_generator(config=data_config)

In [4]:
# Caution: This takes about a minute
scenario = next(data_iter)

2024-10-03 14:37:17.016835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-03 14:37:17.034655: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-10-03 14:38:34.933012: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, wh

### Processing codde

In [8]:
# Write a custom dataloader that loads scenario IDs.
def _preprocess(serialized: bytes) -> dict[str, tf.Tensor]:
    womd_features = dataloader.womd_utils.get_features_description(
        include_sdc_paths=data_config.include_sdc_paths,
        max_num_rg_points=data_config.max_num_rg_points,
        num_paths=data_config.num_paths,
        num_points_per_path=data_config.num_points_per_path,
    )
    womd_features['scenario/id'] = tf.io.FixedLenFeature([1], tf.string)

    deserialized = tf.io.parse_example(serialized, womd_features)
    parsed_id = deserialized.pop('scenario/id')
    deserialized['scenario/id'] = tf.io.decode_raw(parsed_id, tf.uint8)

    return dataloader.preprocess_womd_example(
        deserialized,
        aggregate_timesteps=data_config.aggregate_timesteps,
        max_num_objects=data_config.max_num_objects,
    )

def _postprocess(example: dict[str, tf.Tensor]):
    scenario = dataloader.simulator_state_from_womd_dict(example)
    scenario_id = example['scenario/id']
    return scenario_id, scenario

# Get a single scenario
data_iter = dataloader.get_data_generator(
    data_config, _preprocess, _postprocess
)

Link: https://waymo.com/open/data/motion/tfexample

In [10]:
# Get the unique ID string for a scenario
for scenario_id, _ in data_iter:
    # Decode bytes and return the scenario ID
    scenario_id = scenario_id.tobytes().decode('utf-8')
    print("Scenario ID:", scenario_id)  # Print the scenario ID for the first scenario
    break  


Scenario ID: 20ee2d5990baa725


### Scenario id is not in the metadata

In [11]:
dir(scenario.object_metadata)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'from_tuple',
 'get',
 'ids',
 'is_controlled',
 'is_modeled',
 'is_sdc',
 'is_valid',
 'items',
 'keys',
 'num_objects',
 'object_types',
 'objects_of_interest',
 'replace',
 'shape',
 'to_tuple',
 'validate',
 'values']